In [2]:
pip install downcast

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Importing necessary libraries
from downcast import reduce
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import pickle
import gc
import pickle
import warnings 
warnings.filterwarnings("ignore")

### Loading Dataframes

In [4]:
calendar = pd.read_csv('calendar.csv')

In [5]:
#deleting the rows which has data after 22nd May 2016 i.e, days after 1941 
calendar = calendar[(calendar['date'] <= '2016-05-22')]
#Checking for NULL or Nan values
print("Columns with Null values in calendar dataset ",calendar.columns[calendar.isna().any()].tolist())
#Replacing Nan values with 'no_event' - value
calendar=calendar.fillna('no_event')

Columns with Null values in calendar dataset  ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']


In [6]:
#Loading sales_train_eval dataset
sales_train_eval = pd.read_csv('sales_train_evaluation.csv')

In [7]:
#Loading sell_prices dataset
sell_price = pd.read_csv('sell_prices.csv')

Downcasting the dataframes to reduce the amount of storage used

In [8]:
cal_bfr = calendar.memory_usage(deep=True).sum()
calendar=reduce(calendar)
cal_aftr = calendar.memory_usage(deep=True).sum()
sales_bfr = sales_train_eval.memory_usage(deep=True).sum()
sales_train_eval=reduce(sales_train_eval)
sales_aftr = sales_train_eval.memory_usage(deep=True).sum()
price_bfr = sell_price.memory_usage(deep=True).sum()
sell_price=reduce(sell_price)
price_aftr = sell_price.memory_usage(deep=True).sum()

print("memory usage of calendar dataframe reduced by",cal_aftr/cal_bfr * 100,"percent")
print("memory usage of sales dataframe reduced by",sales_aftr/sales_bfr * 100,"percent")
print("memory usage of price dataframe reduced by",price_aftr/price_bfr * 100,"percent")

memory usage of calendar dataframe reduced by 25.33067651400899 percent
memory usage of sales dataframe reduced by 21.31192592896515 percent
memory usage of price dataframe reduced by 4.797418193536785 percent


### Creating final dataset

In [9]:
#dataframe is pivoted to have all the sales data under a single column
sales_final=sales_train_eval.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 
                                           'store_id', 'state_id'], var_name='d',value_name='sales')

In [10]:
#merging sales data with calendar data to plot total sales per day(in terms of date)
sales_final=sales_final.merge(calendar,on='d',how='left')

In [11]:
#checking for columns with Nan values 
sales_final.columns[sales_final.isna().any()].tolist()

[]

In [12]:
#Merging with price dataframe
sales_final=sales_final.merge(sell_price,on=['wm_yr_wk','item_id','store_id'],how='left')
sales_final.isnull().values.any()

True

In [13]:
#checking for columns with Nan values 
sales_final.columns[sales_final.isna().any()].tolist()

['sell_price']

There are no NULL values before merge with sell_prices dataframe.But after merge we can see NULL values
in 'sell_price' column. This means the particular item is not in stock in that particular store on a 
given day.We will handle NULL values with imputation.

In [14]:
#Median imputation
sales_final['sell_price'].fillna(sales_final.groupby(['item_id','store_id'])['sell_price'].transform('median'),
                              inplace=True)

Due to memory constraints I will be using data from 1st Jan 2014 to train the model. When lag features are added, it is resulting in Nan values in first few rows. 
To avoid computational errors due to Nan values, I will be considering data from 1st October 2014. After lag features are added, I will use the data from 1st Jan 2015.

In [15]:
#deleting the rows which has data before 1st Oct 2014
sales_final = sales_final[(sales_final['date'] >= '2014-10-01')]

In [16]:
#deleting unused dataframes to freeup memory space
del calendar
del sales_train_eval
del sell_price
gc.collect()

0

In [17]:
#Extract number from string
sales_final['d'] = sales_final['d'].str.extract(r"(\d+)").astype(np.int16)

In [18]:
sales_final = sales_final.reset_index(drop=True)

In [19]:
sales_final.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1342,2,2014-10-01,11435,...,10,2014,no_event,no_event,no_event,no_event,1,1,0,8.257812
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1342,0,2014-10-01,11435,...,10,2014,no_event,no_event,no_event,no_event,1,1,0,3.970703
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1342,0,2014-10-01,11435,...,10,2014,no_event,no_event,no_event,no_event,1,1,0,2.970703
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1342,0,2014-10-01,11435,...,10,2014,no_event,no_event,no_event,no_event,1,1,0,4.640625
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1342,0,2014-10-01,11435,...,10,2014,no_event,no_event,no_event,no_event,1,1,0,2.880859


### Feature engineering on calendar dataframe columns

In [20]:
#Removing weekday column and adding is_weekend feature

sales_final["is_Weekend"]=sales_final['wday'].map(lambda x: 1 if x in [1,2] else 0)
sales_final=sales_final.drop("weekday",axis=1)


In [21]:
# Converting snap_CA,snap_WI,snap_TX into one feature named snap

sales_final.loc[sales_final['state_id'] == 'CA', 'snap'] = sales_final.loc[sales_final['state_id'] == 'CA']['snap_CA']
sales_final.loc[sales_final['state_id'] == 'TX', 'snap'] = sales_final.loc[sales_final['state_id'] == 'TX']['snap_TX']
sales_final.loc[sales_final['state_id'] == 'WI', 'snap'] = sales_final.loc[sales_final['state_id'] == 'WI']['snap_WI']
sales_final.drop(['snap_CA','snap_TX','snap_WI'],axis=1,inplace=True)

In [22]:
#adding day of month column
sales_final['day_of_month'] =  sales_final['date'].dt.strftime("%d")
sales_final['day_of_month'] = sales_final['day_of_month'].astype('int')

#### Label encoding categorical features

In [23]:
column = ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in column:
    encoder = LabelEncoder()
    sales_final[feature] = encoder.fit_transform(sales_final[feature])

#### Time series related feature engineering

In [24]:
#Adding lag features
lags = [1,7,28,30]
for lag in tqdm(lags):
    sales_final['lag_'+str(lag)] = sales_final.groupby(['id'],as_index=False)['sales'].shift(lag).astype(np.float16)
              

100%|██████████| 4/4 [00:10<00:00,  2.58s/it]


In [25]:
#Adding rolling window features
window = [7,14,28,35,42]
for i in tqdm(window):
    func = lambda x: x.rolling(i).median()
    sales_final['rolling_median_'+str(i)] = sales_final.groupby(['id'],as_index=False)['sales'].transform(func)

100%|██████████| 5/5 [02:14<00:00, 26.90s/it]


In [26]:
sales_final['rolling_sales_mean'] = sales_final.groupby(['item_id','dept_id',
               'cat_id','store_id','state_id'])['sales'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)

In [27]:
#deleting the rows which has data before 1st Jan 2015
sales_final = sales_final[(sales_final['date'] >= '2015-01-01')]

In [28]:
#checking for columns with Nan values 
sales_final.columns[sales_final.isna().any()].tolist()

[]

In [29]:
#dropping wm_yr_wk column
sales_final.drop('wm_yr_wk',axis=1,inplace=True)
#dropping date column
sales_final = sales_final.drop('date',axis=1)
sales_final = sales_final.reset_index(drop=True)

There are no NULL values in dataset

In [30]:
sales_final = reduce(sales_final)

#### Splitting the dataframe into train and test datasets

Since this is a time series problem, dataset is split on temporal basis instead of random splitting using sklearn

In [31]:
train_set =  sales_final[(sales_final['d'] <= 1885)]
valid_set = sales_final[(sales_final['d'] > 1885) & (sales_final['d'] <= 1913)]
test_set = sales_final[(sales_final['d'] > 1913)]

In [32]:
train_set.to_pickle('X_train.pkl')
valid_set.to_pickle('X_cv.pkl')
test_set.to_pickle('X_test.pkl')